## Extração do Texto das Provas do ENEM

* **Input**: Local da Prova em formato PDF

* **Output**: CSV com as colunas -> *numero_questao*, *enunciado* e *alternativas*

In [1]:
import pymupdf
import re
import csv
import pandas as pd

In [2]:
# Definição de Variáveis de Input e Output
year = 2023

input_path = f"../data/provas/ENEM_{year}.pdf"
output_path = f"../data/extracted_questions/enem_{year}.csv"

In [8]:
# Extraindo texto do ENEM
document = pymupdf.open(input_path)
total_text = ""

for page in document.pages(1, 15):
    text = page.get_text()
    total_text += text + "\n"

In [13]:
# Remover o rodapé
rodape_pattern = r"CN\s+•\s+2º\s+DIA\s+•\s+CADERNO\s+7\s+•\s+AZUL"
total_text = re.sub(rodape_pattern, "", total_text)

# Remover códigos do rodapé, por exemplo: *020325AZ5*
total_text = re.sub(r"\*\w+\*", "", total_text)

# Remover linhas que contenham apenas dígitos (possíveis números de página)
total_text = re.sub(r"\n\s*\d+\s*\n", "\n", total_text)
total_text = re.sub(r"\n\s*\d+\s*$", "", total_text)

# Dividindo o texto em blocos de questões
question_blocks = re.split(r"QUESTÃO\s+", total_text)
question_blocks = [block.strip() for block in question_blocks if block.strip()]

results = []

for block in question_blocks:
    # Extraindo número da questão
    m = re.match(r"(\d+)", block)
    if not m:
        continue 
    num_quest = m.group(1)
    
    content = block[m.end():].strip()
    
    # Separando enunciado e alternativas.
    parts = re.split(r"\nA\s*\n", content, maxsplit=1)
    if len(parts) == 2:
        enunciado = parts[0].strip()
        alt_text = "A\n" + parts[1].strip()
    else:
        enunciado = content
        alt_text = ""
    
    # Extraindo alternativas
    alt_regex = r"([A-E])\s+[A-E]\s+(.*?)(?=(?:\n[A-E]\s+[A-E]\s+)|\Z)"
    alt_raw = re.findall(alt_regex, alt_text, flags=re.DOTALL)
    
    # Formatando alternativas como "A: texto"
    alternativas = "; ".join([f"{letter}: {text.strip()}" for letter, text in alt_raw])
    
    results.append({
        "numero_questao": num_quest,
        "enunciado": enunciado,
        "alternativas": alternativas
    })

In [14]:
# Salvando em CSV
with open(output_path, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["numero_questao", "enunciado", "alternativas"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in results[1:]:
        writer.writerow(row)

print("Extração Finalizada")

Extração Finalizada


In [15]:
df = pd.read_csv(output_path)
df.head()

,numero_questao,enunciado,alternativas
0,92,- Ciências da Natureza e suas Tecnologias\nO s...,NaN
1,93,- Ciências da Natureza e suas Tecnologias\nO p...,NaN
2,94,- Ciências da Natureza e suas Tecnologias\nA f...,NaN
3,95,- Ciências da Natureza e suas Tecnologias\nOs ...,NaN
4,96,- Ciências da Natureza e suas Tecnologias\nA s...,NaN


## Enem Digital (2021)

In [17]:
# Definição de Variáveis de Input e Output
year = 2021

input_path = f"../data/provas/ENEM_{year}.pdf"
output_path = f"../data/extracted_questions/enem_{year}.csv"

In [22]:
# Extraindo texto do ENEM
document = pymupdf.open(input_path)
total_text = ""

for page in document.pages(5, 53):
    text = page.get_text()
    total_text += text + "\n"

In [23]:
# Padrão do ENEM Digital
question_pattern = r"Questão\s+(\d+)\s+-\s+Ciências da Natureza e suas Tecnologias(.*?)(?=Questão\s+\d+\s+-\s+Ciências da Natureza e suas Tecnologias|$)"
question_matches = re.findall(question_pattern, total_text, flags=re.DOTALL)

results = []

# Padrão de formatação das alternativas do ENEM Digital
alt_regex = r"\n(?:\d+\n)*([A-E])\n([^\n]+)"

for num_quest, block in question_matches:
    content = block.strip()

    alt_raw = re.findall(alt_regex, content)

    enunciado = content
    alt_match = re.search(alt_regex, content)
    if alt_match:
        enunciado = content[: alt_match.start()].strip()

    # Formatando alternativas como "A: texto"
    alternativas = "; ".join([f"{letter}: {text.strip()}" for letter, text in alt_raw])

    results.append(
        {
            "numero_questao": num_quest,
            "enunciado": enunciado,
            "alternativas": alternativas,
        }
    )

In [24]:
# Salvando em CSV
with open(output_path, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["numero_questao", "enunciado", "alternativas"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in results[1:]:
        writer.writerow(row)

print("Extração Finalizada")

Extração Finalizada


In [25]:
df = pd.read_csv(output_path)
df.head()

,numero_questao,enunciado,alternativas
0,92,O sino dos ventos é composto por várias barras...,A: f < f e v < v .; B: f < f e v = v .; ...
1,93,O polvo mimético apresenta padrões cromáticos ...,A: originaram-se do mesmo ancestral que esses ...
2,94,A figura foi extraída de um antigo jogo para c...,A: 30 m/s.; B: 35 m/s.; C: 40 m/s.; D: 45 m/s....
3,95,Os pesticidas organoclorados foram amplamente ...,"A: 3,9 g.; B: 31,2 g.; C: 62,5 g.; D: 125,0 g...."
4,96,A sequência de nucleotídeos do RNA mensageiro ...,A: Deleção de G no códon 3.; B: Substituição d...
